# 🌙 QuranLab AI: Complete Demo

> **Ethical AI for Quranic Recitation Analysis**  
> Part of the [ADANiD Ecosystem](https://github.com/ADANiD-AI/adan-id-opencloud)

## 📋 Features
- **ASR**: Transcribe Quranic Arabic
- **Tajweed**: Detect errors (madd, ghunnah, idgham)
- **Position**: Identify Surah & Ayah
- **Scoring**: Pronunciation quality (0–100)
- **Abjad Validation**: Numerical fingerprint verification
- **Jannah Points**: Reward system for accuracy

In [ ]:
# Install dependencies
!pip install -q torch transformers librosa soundfile huggingface-hub pandas numpy matplotlib seaborn

In [ ]:
# Import libraries
import os
import sys
import torch
import librosa
import soundfile as sf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML

# Add project to path
sys.path.append('../projects/quranlab/ai')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

## 🔑 Hugging Face Authentication

In [ ]:
# Set HF token (use GitHub Secrets in production)
os.environ["HF_TOKEN"] = "your_hf_token_here"  # ← Replace with your token

# Verify authentication
from huggingface_hub import whoami
try:
    user_info = whoami()
    print(f"✅ Logged in as: {user_info['name']}")
except Exception as e:
    print(f"❌ Authentication failed: {e}")

## 📊 Load Abjad Dataset

In [ ]:
# Load Abjad dataset
abjad_df = pd.read_csv('../projects/quranlab/data/quran_abjad.csv')
print(f"✅ Loaded {len(abjad_df)} verses from Abjad dataset")
display(abjad_df.head(5))

## 🧠 Abjad Calculator Class

In [ ]:
class NoorAbjadCalculator:
    def __init__(self, dataframe):
        self.df = dataframe
        self.abjad_map = {
            'ا': 1, 'ب': 2, 'ج': 3, 'د': 4, 'ه': 5,
            'و': 6, 'ز': 7, 'ح': 8, 'ط': 9, 'ي': 10,
            'ك': 20, 'ل': 30, 'م': 40, 'ن': 50, 'س': 60,
            'ع': 70, 'ف': 80, 'ص': 90, 'ق': 100,
            'ر': 200, 'ش': 300, 'ت': 400, 'ث': 500, 'خ': 600,
            'ذ': 700, 'ض': 800, 'ظ': 900, 'غ': 1000
        }
    
    def calculate_abjad(self, text: str) -> int:
        total = 0
        for char in text:
            if char in self.abjad_map:
                total += self.abjad_map[char]
        return total
    
    def lookup_verse(self, surah: int, ayah: int) -> dict:
        result = self.df[(self.df['surah'] == surah) & (self.df['ayah'] == ayah)]
        if len(result) > 0:
            row = result.iloc[0]
            return {
                'text': row['arabic_text'],
                'abjad_sum': row['abjad_sum'],
                'jannah_points': row['jannah_points']
            }
        return None

## 🎙️ QuranLab AI Inference Pipeline

In [ ]:
# Import the main pipeline
from quran_inference_pipeline import run_full_inference_pipeline

# Create test audio if not exists
test_audio_path = '../projects/quranlab/ai/test.wav'
if not os.path.exists(test_audio_path):
    print("⚠️ Test audio not found. Creating dummy audio...")
    duration = 2
    sample_rate = 16000
    silent_audio = np.zeros(int(sample_rate * duration))
    sf.write(test_audio_path, silent_audio, sample_rate)

# Run inference
print("🚀 Running QuranLab AI inference pipeline...")
result = run_full_inference_pipeline(test_audio_path)

if result:
    display(HTML(f"<h3>✅ Inference Results</h3>"))
    print(f"Text: {result['text']}")
    print(f"Position: Surah {result['surah']}, Ayah {result['ayah']}")
    print(f"Tajweed Errors: {result['tajweed_errors']}")
    print(f"Score: {result['pronunciation_score']}/100")

## 🔍 Abjad Validation

In [ ]:
# Initialize Abjad calculator
abjad_calc = NoorAbjadCalculator(abjad_df)

# Validate recitation
if result and result['text']:
    expected = abjad_calc.lookup_verse(result['surah'], result['ayah'])
    if expected:
        user_abjad = abjad_calc.calculate_abjad(result['text'])
        valid = user_abjad == expected['abjad_sum']
        
        display(HTML(f"<h3>🔍 Abjad Validation</h3>"))
        print(f"User Abjad: {user_abjad}")
        print(f"Expected: {expected['abjad_sum']}")
        print(f"Valid: {valid}")
        print(f"Jannah Points: {expected['jannah_points']}")

## 📈 Dataset Analysis

In [ ]:
# Basic analysis
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Abjad sum distribution
axes[0].hist(abjad_df['abjad_sum'], bins=50, alpha=0.7, color='skyblue')
axes[0].set_title('Abjad Sum Distribution')
axes[0].set_xlabel('Abjad Sum')
axes[0].set_ylabel('Frequency')

# Jannah Points distribution
axes[1].hist(abjad_df['jannah_points'], bins=20, alpha=0.7, color='lightcoral')
axes[1].set_title('Jannah Points Distribution')
axes[1].set_xlabel('Jannah Points')
axes[1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

## 🔗 Integration Points

| Platform | Repository |
|----------|------------|
| **Hugging Face Models** | [ADANiD/Quranlab](https://huggingface.co/ADANiD/Quranlab) |
| **Hugging Face Dataset** | [ADANiD/Quranlab-islamic-dataset](https://huggingface.co/datasets/ADANiD/Quranlab-islamic-dataset) |
| **Kaggle Dataset** | [adnanmd76/nooreabjad](https://www.kaggle.com/datasets/adnanmd76/nooreabjad) |
| **GitHub** | [ADANiD-AI/adan-id-opencloud](https://github.com/ADANiD-AI/adan-id-opencloud) |

> "And We have certainly made the Qur’an easy for remembrance..." (Quran 54:17)